In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import os

In [ ]:
"""
Implementation of the VQE algorithm to find the optimum bond-length of carbon monoxide (CO).
"""

In [ ]:
# Atom symbols for CO
symbols = ["C", "O"]

# Define coordinates as a function of bond length
def generate_coordinates(bond_length):
    return np.array([
        [0.0, 0.0, 0.0],             # Carbon at origin
        [0.0, 0.0, bond_length]      # Oxygen placed along z-axis
    ])

In [ ]:
# Bond scan range
bond_lengths = np.linspace(0.9, 1.5, 5)  # In Ångströms
energies = []

for bond_length in bond_lengths:
    print(f"Running VQE for bond length: {bond_length:.2f} Å")

    # Define the water Hamiltonian and the number of qubits required
    coordinates = generate_coordinates(bond_length)
    hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0)
    electrons = 14  # 6 from C, 8 from O

    hf = qchem.hf_state(electrons=electrons, orbitals=qubits)
    singles, doubles = excitations(electrons, qubits)
    num_wires = qubits

    dev = qml.device("default.qubit", wires=num_wires)

    def ansatz(params_singles, params_doubles):
        qml.BasisState(hf, wires=range(num_wires))
        for i, excitation in enumerate(singles):
            qml.SingleExcitation(params_singles[i], wires=excitation)
        for i, excitation in enumerate(doubles):
            qml.DoubleExcitation(params_doubles[i], wires=excitation)

    @qml.qnode(dev)
    def cost_function(params_singles, params_doubles):
        ansatz(params_singles, params_doubles)
        return qml.expval(hamiltonian)

    # Adaptive Moment Estimation (Adam) optimizer
    opt = qml.AdamOptimizer(stepsize=0.4)
    theta_s = np.zeros(len(singles), requires_grad=True)
    theta_d = np.zeros(len(doubles), requires_grad=True)

    for _ in range(50):
        (theta_s, theta_d), _ = opt.step_and_cost(cost_function, theta_s, theta_d)

    energy = cost_function(theta_s, theta_d)
    print(f"  → Final Energy: {energy:.6f} Ha\n")
    energies.append(energy)

In [ ]:
plt.plot(bond_lengths, energies, marker='o')
plt.xlabel("C–O Bond Length (Å)")
plt.ylabel("Ground State Energy (Ha)")
plt.title("Potential Energy Curve for CO")
plt.grid(True)

# Ensure the "images" directory exists
os.makedirs('images', exist_ok=True)

# Save the figure to the "images" directory
plt.savefig("images/CO_Optimal_Bond_Length.png")
plt.show()